# TensorFlow.jl
## Jon Malmaud

6/21/17
JuliaCon 2017

In [14]:
Pkg.add("TensorFlow")
using TensorFlow
const tf = TensorFlow

TensorFlow

In [10]:
session = Session()

Session(Ptr{Void} @0x000000011b8ba370)

# Organization
* Why TensorFlow?
* How do you use TensorFlow?
* How TensorFlow is implemented
* Time allowing - work through a real example

# Why TensorFlow?
* I will show you

# Why TensorFlow.jl?

* Multiple Dispatch
* Macros
* JIT

# Static graph framework

In [60]:
x = placeholder(Float32)
y = x + 1
run(session, y, Dict(x=>3))

4.0f0

In [61]:
tf.set_def_graph(Graph())
@tf begin
    x = constant(2.0)
    y = x+1
    z = sin(y)
end


<Tensor z:1 shape=() dtype=Float64>

In [64]:
sin(2+1)

0.1411200080598672

In [62]:
visualize()

In [63]:
train.export_meta_graph("/Users/malmaud/graph");

# Multiple dispatch

In [54]:
f(x) = sin(exp(x))

f (generic function with 1 method)

In [65]:
f(2.0)

0.8938549549128102

In [66]:
result =  f(constant(2.0))
run(session, result)

0.8938549549128102

# Control flow - Sum of squares

In [22]:
sum((0:10).^2)

385

## Ugly

In [67]:
session = Session(Graph())
i = constant(0)
result = constant(0)
_, result_final = while_loop(
    (i, result)-> i≤10, 
    (i,result)-> [i+1, result+i^2], 
    [i, result])
run(session, result_final)

385

## Pretty

In [68]:
session = Session(Graph())
i = constant(0)
result = constant(0)
i_final, result_final = @tf while i≤10
    sq = i^2
    [i=>i+1, result=>result+sq]
end
run(session, result_final)

385

In [73]:
x = constant(randn(2,3))

<Tensor Const_146:1 shape=(2, 3) dtype=Float64>

In [77]:
run(session, x[1,2:3])

2-element Array{Float64,1}:
 0.725545
 0.933776

# Accessing TensorFlow ops

* 623 operations
* eg FakeQuantWithMinMaxVarsPerChannelGradient


In [70]:
?Ops.sin

```
 sin(x)
```

Computes sin of x element-wise.


# Example model

In [72]:
using TensorFlow
using Distributions
include(joinpath(Pkg.dir("TensorFlow"), "examples", "mnist_loader.jl"))

loader = DataLoader()

session = Session(Graph())

function weight_variable(shape)
    initial = map(Float32, rand(Normal(0, .001), shape...))
    return Variable(initial)
end

function bias_variable(shape)
    initial = fill(Float32(.1), shape...)
    return Variable(initial)
end

function conv2d(x, W)
    nn.conv2d(x, W, [1, 1, 1, 1], "SAME")
end

function max_pool_2x2(x)
    nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], "SAME")
end

@tf begin

    x = placeholder(Float32)
    y_ = placeholder(Float32)

    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])

    x_image = reshape(x, [-1, 28, 28, 1])

    h_conv1 = nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])

    h_conv2 = nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    W_fc1 = weight_variable([7*7*64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = nn.relu(h_pool2_flat * W_fc1 + b_fc1)

    keep_prob = placeholder(Float32)
    h_fc1_drop = nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    y_conv = nn.softmax(h_fc1_drop * W_fc2 + b_fc2)

    cross_entropy = mean(-sum(y_.*log(y_conv), axis=[2]))
end

train_step = train.minimize(train.AdamOptimizer(1e-4), cross_entropy)

correct_prediction = indmax(y_conv, 2) .== indmax(y_, 2)

accuracy = mean(cast(correct_prediction, Float32))

run(session, global_variables_initializer())

summaries = [tf.summary.scalar("accuracy", accuracy), tf.summary.scalar("Cross entropy", cross_entropy)]
summary_op = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter("/Users/malmaud/tmp/summaries")

for i in 1:100
    batch = next_batch(loader, 50)
    if i%100 == 1
        train_accuracy = run(session, accuracy, Dict(x=>batch[1], y_=>batch[2], keep_prob=>1.0))
        info("step $i, training accuracy $train_accuracy")
    end
    summary_value, _ = run(session, [summary_op, train_step], Dict(x=>batch[1], y_=>batch[2], keep_prob=>.5))
    write(summary_writer, summary_value, i)
end

testx, testy = load_test_set()
test_accuracy = run(session, accuracy, Dict(x=>testx, y_=>testy, keep_prob=>1.0))
info("test accuracy $test_accuracy")


ERROR:tensorflow:Unable to get size of /Users/malmaud/tmp/summaries/events.out.tfevents.1498090700.maltop: /Users/malmaud/tmp/summaries/events.out.tfevents.1498090700.maltop
INFO: step 1, training accuracy 0.1
ERROR:tensorflow:Unable to get size of /Users/malmaud/tmp/summaries/events.out.tfevents.1498090700.maltop: /Users/malmaud/tmp/summaries/events.out.tfevents.1498090700.maltop
INFO: test accuracy 0.894


In [43]:
visualize(summary_writer)

# Thanks!

* Oxinabox (Lyndon White)
* Google Brain members
* Other contributors